# KoBERT finetuning

In [1]:
#!pip install ipywidgets  # for vscode
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-75nb5_br
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-75nb5_br


In [7]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm.notebook import tqdm

In [2]:
from kobert import get_tokenizer
from kobert import get_pytorch_kobert_model

In [3]:
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

In [4]:
import pandas as pd

In [5]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [28]:
## CPU
#device = torch.device("cpu")

## GPU
device = torch.device("cuda:0")

In [9]:
bertmodel, vocab = get_pytorch_kobert_model(cachedir=".cache")

using cached model. /content/.cache/kobert_v1.zip
using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [10]:
!wget -O .cache/ratings_train.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
!wget -O .cache/ratings_test.txt http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt

--2022-05-26 17:00:53--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_train.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.144.59
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.144.59|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 14628807 (14M) [text/plain]
Saving to: ‘.cache/ratings_train.txt’

.cache/ratings_trai 100%[===================>]  13.95M  6.36MB/s    in 2.2s    

2022-05-26 17:00:56 (6.36 MB/s) - ‘.cache/ratings_train.txt’ saved [14628807/14628807]

--2022-05-26 17:00:57--  http://skt-lsl-nlp-model.s3.amazonaws.com/KoBERT/datasets/nsmc/ratings_test.txt
Resolving skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)... 52.219.144.59
Connecting to skt-lsl-nlp-model.s3.amazonaws.com (skt-lsl-nlp-model.s3.amazonaws.com)|52.219.144.59|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 4893335 (

In [11]:
dataset_train = pd.read_csv('/content/drive/MyDrive/capstone/감성대화말뭉치(최종데이터)_Training.csv')
dataset_validation = pd.read_csv('/content/drive/MyDrive/capstone/감성대화말뭉치(최종데이터)_Validation.csv')


dataset = pd.concat([dataset_train, dataset_validation])

dataset['대화_내용'] = dataset[['사람문장1','시스템응답1','사람문장2','시스템응답2','사람문장3','시스템응답3','사람문장4','시스템응답4']].apply(lambda row: ' '.join(row.values.astype(str)), axis=1)
dataset = dataset.dropna(subset = ['번호'])

dataset= dataset.drop( ['번호','사람문장1','시스템응답1','사람문장2','시스템응답2','사람문장3','시스템응답3','사람문장4','시스템응답4'],axis=1) 

#dataset.count() -> 40800 + 5100 = 대략 46000개
#dataset.tail()

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2882: DtypeWarning: Columns (1,2,3,4,5,6,7,8,9) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [12]:
dataset.count()

감정_대분류    46009
대화_내용     46009
dtype: int64

In [14]:
data1 = dataset.loc[dataset['감정_대분류'] == '기쁨']
data2 = dataset.loc[dataset['감정_대분류'] == '불안']
data3 = dataset.loc[dataset['감정_대분류'] == '슬픔']
data4 = dataset.loc[dataset['감정_대분류'] == '당황']
data5 = dataset.loc[dataset['감정_대분류'] == '상처']
data6 = dataset.loc[dataset['감정_대분류'] == '분노']

new_data = data1.append([data2, data3, data4, data5, data6], sort=False)
new_data = pd.DataFrame(new_data)
new_data.head()

,감정_대분류,대화_내용
0,기쁨,아내가 드디어 출산하게 되어서 정말 신이 나. 아내분이 출산을 하시는군요. 정말 축...
3,기쁨,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. ...
4,기쁨,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 기분 좋으시겠어요. 앞으로는 어...
14,기쁨,남편이 죽기 전에 손자들 얼굴 보고 싶다고 했는데 다행히 늦지 않아서 안도했어. 남...
23,기쁨,친구가 사준 로또가 당첨이 됐어. 축하드려요. 당첨금은 어떻게 하실 예정인가요? 로...


In [15]:
from sklearn.preprocessing import LabelEncoder

encoder = LabelEncoder()
encoder.fit(new_data['감정_대분류'])
new_data['감정_대분류'] = encoder.transform(new_data['감정_대분류'])
new_data.head()

,감정_대분류,대화_내용
0,0,아내가 드디어 출산하게 되어서 정말 신이 나. 아내분이 출산을 하시는군요. 정말 축...
3,0,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해. ...
4,0,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어. 기분 좋으시겠어요. 앞으로는 어...
14,0,남편이 죽기 전에 손자들 얼굴 보고 싶다고 했는데 다행히 늦지 않아서 안도했어. 남...
23,0,친구가 사준 로또가 당첨이 됐어. 축하드려요. 당첨금은 어떻게 하실 예정인가요? 로...


In [16]:
mapping = dict(zip(range(len(encoder.classes_)), encoder.classes_))
mapping

{0: '기쁨', 1: '당황', 2: '분노', 3: '불안', 4: '상처', 5: '슬픔'}

In [17]:
from sklearn.model_selection import train_test_split
train, test = train_test_split(new_data, test_size=0.2, random_state=42)
print("train shape is:", len(train))
print("test shape is:", len(test))


train shape is: 36805
test shape is: 9202


In [18]:
train = train.to_numpy()
test = test.to_numpy()

In [19]:
train[0]

array([3,
       '일을 하는데 팀원들이 모두 나를 비난해서 앞으로 직장을 다닐 때 불편할 것 같아. 팀원들이 비난을 해서 앞으로 직장이 걱정되시는군요. 팀원들과 소통하지 않고 혼자 일을 처리해서 그런 것 같아. 어떻게 대처하는 것이 좋을까요? 사과를 하고 앞으로 바뀐 모습을 보여줘야겠어. 팀원들과 다시 잘 지내게 되면 좋겠네요. nan nan'],
      dtype=object)

In [20]:
## Setting parameters
max_len = 64
batch_size = 64
warmup_ratio = 0.1
num_epochs = 5
max_grad_norm = 1
log_interval = 200
learning_rate =  5e-5

In [21]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(
            bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))


In [22]:
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

using cached model. /content/.cache/kobert_news_wiki_ko_cased-1087f8699e.spiece


In [23]:
data_train = BERTDataset(train, 1, 0 , tok, max_len, True, False)
data_test = BERTDataset(test, 1, 0, tok, max_len, True, False)

In [24]:
data_train[0]

(array([   2, 3817, 4931, 4797, 7020, 5940, 2047, 1370, 6116, 2521, 7850,
        3192, 4355, 7088, 1562, 5777, 1844, 2499, 7836,  905,  832,  517,
          54, 4797, 7020, 5940, 2521, 7088, 5007, 3192, 4355, 7096,  880,
        5886, 6705, 5760, 5512, 6999,  517,   54, 4797, 7020, 5932, 2850,
        7819, 3149, 5102, 3817, 4465, 7850, 1201,  905,  832,  517,   54,
        3225, 1664, 7794,  912, 4204, 7088, 5591, 6999,    3], dtype=int32),
 array(64, dtype=int32),
 array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
        0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
       dtype=int32),
 3)

In [25]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size, num_workers=6)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size, num_workers=6)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 6 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


In [26]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=6,
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device))
        if self.dr_rate:
            out = self.dropout(pooler)
        else:
            out = pooler
        return self.classifier(out)

In [ ]:
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

In [ ]:
# Prepare optimizer and schedule (linear warmup and decay)
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.01},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

In [ ]:
optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

In [ ]:
t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

In [ ]:
scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

In [ ]:
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(train_dataloader), total=len(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in tqdm(enumerate(test_dataloader), total=len(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

In [ ]:
new_data_df = pd.read_csv('/content/drive/MyDrive/capstone/2022-05-23 05_22_28.963275.csv', sep=",", engine='python', encoding ="utf-8")
new_data_df.head()

new_data = new_data_df.to_numpy()

In [ ]:
def predict(lyrics):

    data = [lyrics, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size, num_workers=6)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("기쁨")
            elif np.argmax(logits) == 1:
                test_eval.append("불안")
            elif np.argmax(logits) == 2:
                test_eval.append("슬픔")
            elif np.argmax(logits) == 3:
                test_eval.append("당황")
            elif np.argmax(logits) == 4:
                test_eval.append("상처")
            elif np.argmax(logits) == 5:
                test_eval.append("분노")
            
            return test_eval[0]


In [ ]:
end = len(new_data_df)
tmp = []

for i in range(0,end):
  lyrics = new_data[i][2]
  result = predict(lyrics)

  tmp.append(result)

  print(result)


In [ ]:
print(tmp)
len(tmp)

In [ ]:
new_data_df['감정']=tmp

In [ ]:
new_data_df.head()

In [ ]:
import xlwt, xlrd
from google.colab import files

# 해당이름의 csv파일을 읽어옴
#r_csv = pd.read_csv("/content/drive/MyDrive/capstone/2022-05-23 05_22_28.963275.csv")


# 저장할 xlsx파일의 이름을 정함
save_xlsx = pd.ExcelWriter("FileName.xlsx")
new_data_df.to_excel(save_xlsx, index = False) # xlsx 파일로 변환

save_xlsx.save() #xlsx 파일로 저장

files.download(save_xlsx)